In [1]:
import pandas as pd
import sys
import numpy as np

sys.path.append('~/PycharmProjects/Boston_housing/')

df_master = pd.read_csv('./data/' + 'redfin_2020-01-19-08-17-26.csv')
df_master.pop('SOLD DATE')
df_condo = pd.read_csv('./data/' + 'Boston_condo_transaction.csv',index_col=0)
df_sfr = pd.read_csv('./data/' + 'Boston_single_family_residential_transaction.csv',index_col=0)
df_townhouse = pd.read_csv('./data/' + 'Boston_townhouse_transaction.csv',index_col=0)

df_transaction = pd.concat([df_condo,df_sfr,df_townhouse])

df = df_master.join(df_transaction,how='inner')

def outlier(x):
    return 1.5*(np.quantile(x,0.75)-np.quantile(x,0.25)) + np.quantile(x,0.75)

df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])
df['LIST DATE'] = pd.to_datetime(df['LIST DATE'])
df['DAYS ON MKT'] = (df['SOLD DATE'] - df['LIST DATE']).apply(lambda x: x.days)
df['HOA/MONTH'] = df['HOA/MONTH'].fillna(0)

df_outlier = df[['DAYS ON MKT','SOLD PRICE','LIST PRICE','HOA/MONTH']].apply(lambda x: outlier(x),axis=0)

print('number of data before outlier removal: ',len(df))

df = df[(df['DAYS ON MKT'] < df_outlier['DAYS ON MKT'])&
        (df['SOLD PRICE'] < df_outlier['SOLD PRICE']) &
        (df['LIST PRICE'] < df_outlier['LIST PRICE']) &
        (df['HOA/MONTH'] < df_outlier['HOA/MONTH'])]

print('number of data after outlier removal: ',len(df))

df = df[df['DAYS ON MKT'] > 0] # price history scraped after the spreadsheet

print('number of data after removing new data: ',len(df))

number of data before outlier removal:  7912
number of data after outlier removal:  6357
number of data after removing new data:  6350


In [7]:
import matplotlib.pyplot as plt
%matplotlib inline


import seaborn as sns

import geopandas

sns.set()


gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(x=df.LONGITUDE, y=df.LATITUDE)
)


